# 03 — Model Conversion to TensorFlow.js

Цей ноутбук:
- бере `artifacts/model.h5`
- конвертує в TF.js формат у `web/model/`
- перевіряє, що файли з’явилися:
  - `web/model/model.json`
  - `web/model/group1-shard1of1.bin`

Посилання:
- TensorFlow.js: https://www.tensorflow.org/js

In [ ]:
import os, subprocess, sys
from pathlib import Path

MODEL_H5 = Path("../artifacts/model.h5")
OUT_DIR = Path("../web/model")
OUT_DIR.mkdir(parents=True, exist_ok=True)

assert MODEL_H5.exists(), f"Не знайдено {MODEL_H5}. Спочатку запусти 02_model_training.ipynb"

print("Input:", MODEL_H5)
print("Output dir:", OUT_DIR)

In [ ]:
# Конвертація через tensorflowjs_converter
# Якщо команда не знайдена — встанови пакет: pip install tensorflowjs

cmd = [
    sys.executable, "-m", "tensorflowjs.converters.converter",
    "--input_format", "keras",
    str(MODEL_H5),
    str(OUT_DIR)
]

print("Running:", " ".join(cmd))
proc = subprocess.run(cmd, capture_output=True, text=True)
print(proc.stdout)
print(proc.stderr)
if proc.returncode != 0:
    raise RuntimeError("Conversion failed. See logs above.")

In [ ]:
# Перевірка артефактів
files = sorted([p.name for p in OUT_DIR.glob("*")])
files

In [ ]:
# Мінімальна перевірка: model.json існує і shard існує
assert (OUT_DIR / "model.json").exists(), "Нема web/model/model.json"
shards = list(OUT_DIR.glob("*.bin"))
assert len(shards) >= 1, "Нема .bin shard файлів"
print("OK ✅ TF.js model artifacts are ready:", shards)